In [1]:
import numpy as np
import pandas as pd

In [2]:
# Пункт 2
ff = pd.read_csv('SMSSpamCollection.txt', sep='\t', names=['label','text'])

In [3]:
# Пункт 3
labels = ff['label'].values
texts = ff['text'].values
lab_codes = np.array(map(lambda x: 1 if x == 'spam' else 0, labels))

In [4]:
# Пункт 4
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer() 

X = vec.fit_transform(texts)

In [5]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression

C:\Users\Kirill\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
# Пункт 5
print np.mean(cross_val_score(LogisticRegression(), X, lab_codes, cv=10, scoring='f1'))

0.932640298361


Пункт 6

In [7]:
clf = LogisticRegression().fit(X, lab_codes)

In [8]:
X_t = ["FreeMsg:	Txt:	CALL	to	No:	86888	&	claim	your	reward	of	3	hours	talk	time	to	use	from	your	phone	now!	Subscribe6GB",
"FreeMsg:	Txt:	claim	your	reward	of	3	hours	talk	time",
"Have	you	visited	the	last	lecture	on	physics?",
"Have	you	visited	the	last	lecture	on	physics?	Just	buy	this	book	and	you	will	have	all	materials!	Only	99$",
"Only	99$"]

In [9]:
# Ответ пункта 6
X_test = vec.transform(X_t)
print " ".join(map(str, clf.predict(X_test)))

1 1 0 0 0


Пункт 7

In [10]:
output = cross_val_score(LogisticRegression(), CountVectorizer(ngram_range=(2,2)).fit_transform(texts), lab_codes, scoring='f1').mean()
print "%.2f" % output

0.78


In [11]:
output = cross_val_score(LogisticRegression(), CountVectorizer(ngram_range=(3,3)).fit_transform(texts), lab_codes, scoring='f1').mean()
print "%.2f" % output

0.67


In [12]:
output = cross_val_score(LogisticRegression(), CountVectorizer(ngram_range=(1,3)).fit_transform(texts), lab_codes, scoring='f1').mean()
print "%.2f" % output

0.91


Пункт 8

In [13]:
from sklearn.naive_bayes import MultinomialNB

In [14]:
print cross_val_score(MultinomialNB(), CountVectorizer(ngram_range=(2,2)).fit_transform(texts), lab_codes, scoring='f1').mean()
print cross_val_score(MultinomialNB(), CountVectorizer(ngram_range=(3,3)).fit_transform(texts), lab_codes, scoring='f1').mean()
print cross_val_score(MultinomialNB(), CountVectorizer(ngram_range=(1,3)).fit_transform(texts), lab_codes, scoring='f1').mean()

0.676920431076
0.405096054216
0.909900868195


Пункт 9

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

print cross_val_score(MultinomialNB(), TfidfVectorizer().fit_transform(texts), lab_codes, scoring='f1').mean()

0.797003520966


In [16]:
from sklearn.pipeline import Pipeline

In [17]:
pipeline = Pipeline([('vect', TfidfVectorizer()), ('clf', LogisticRegression())])
print cross_val_score(pipeline, texts, lab_codes, scoring='f1').mean()

0.866600422941


Пункт 10

In [41]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([('vect', CountVectorizer()), ('clf', LogisticRegression())])

tuned_parameters = [{'vect__binary': [True, False]}]

clf = GridSearchCV(pipeline, tuned_parameters, scoring='f1')

In [42]:
clf.fit(texts, lab_codes)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'vect__binary': [True, False]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [43]:
clf.grid_scores_

C:\Users\Kirill\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92726, std: 0.00685, params: {'vect__binary': True},
 mean: 0.92559, std: 0.00164, params: {'vect__binary': False}]

## Выводы
Собственно какие выводы можно сделать:
1. CountVectorizer обгоняет MultinomialNB.
2. CountVectorizer лучше работает без n-грамм вообще. Попробую предположить, что это из-за того, что в спаме как правило нет длинных словосочетаний. Всё ограничивается некими специфическими для спама словами. К примеру, глаголы: "Спешите!", "Купите" т.п.
3. То, что мы запускаем TfidfVectorizer на всех текстах вносит свой вклад в переобучение, ухудшая результат на кросс-валидации.